<a href="https://colab.research.google.com/github/retroam/harmonizer/blob/develop/ocean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q langchain deeplake openai tiktoken anthropic python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.6/934.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 123.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 3.8 MB/s eta 0:00:0

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv

In [ ]:
fl = 'OpenClimate/harmonize/data/raw/BP_review/bp-stats-review-2022-all-data.xlsx'
fl = os.path.abspath(fl)

In [ ]:
def df_wide_to_long(df=None,
                    value_name=None,
                    var_name=None):

    # set default values (new column names)
    # new column name with {value_vars}
    var_name = "year" if var_name is None else var_name
    # new column name with values
    value_name = "values" if value_name is None else value_name

    # ensure correct type
    assert isinstance(df, pd.core.frame.DataFrame), f"df must be a DataFrame"
    assert isinstance(var_name, str), f"var_name must a be string"
    assert isinstance(value_name, str), f"value_name must be a string"

    # ensure column names are strings
    df.columns = df.columns.astype(str)

    # columns to use as identifiers (columns that are not number)
    id_vars = [val for val in list(df.columns) if not val.isdigit()]

    # columns to unpivot (columns that are numbers)
    value_vars = [val for val in list(df.columns) if val.isdigit()]

    # Unpivot (melt) a DataFrame from wide to long format
    df_long = df.melt(id_vars=id_vars,
                      value_vars=value_vars,
                      var_name=var_name,
                      value_name=value_name)

    # convert var_name column to int
    df_long[var_name] = df_long[var_name].astype(int)

    return df_long

In [ ]:
import pycountry
from typing import List
from typing import Dict


def country_lookup(name):
    try:
        return pycountry.countries.lookup(name).alpha_2
    except LookupError:
        return float('NaN')

In [ ]:
raw = pd.read_excel(fl, sheet_name='CO2e Emissions', header=2)

In [ ]:
xl = pd.ExcelFile(fl)
sheets = xl.sheet_names

replace_dict = {
    'Trinidad & Tobago': 'Trinidad and Tobago',
    'China Hong Kong SAR': 'China',
    'Iran': 'Iran, Islamic Republic of'
}

drop_columns = ['2021.1', '2011-21', '2021.2']
rename_columns = {'Million tonnes of carbon dioxide equivalent': 'country'}
country_groups = ['Central America', 'Eastern Africa', 'Middle Africa', 'Western Africa']
not_countries = ["Source:", "Notes:", "Growth", "Data ", "European Union", "OECD", "0.05%", "Other "]

df = (
    pd.read_excel(fl, sheet_name='CO2e Emissions', header=2)
    .drop(columns = drop_columns)
    .rename(columns = rename_columns)
    .loc[lambda x: x['country'].notnull()]
    .loc[lambda x: ~x['country'].str.contains('total', case=False)]
    .loc[lambda x: ~x['country'].isin(country_groups)]
    .loc[lambda x:  ~(x['country'].str.contains('|'.join(not_countries)))]
    .assign(country = lambda x: x.country.replace(replace_dict))
)

# reshape dataframe
df = df_wide_to_long(df, value_name='emissions', var_name='year')

# convert from million metric tones to metric tonnes
df['total_emissions'] = df['emissions'].apply(lambda x: x * 10**6)

# get actor_id from country name
df_iso = pd.DataFrame(
    data=[(name, country_lookup(name)) for name in set(list(df.country))],
    columns=['country', 'actor_id']
)

if not_found:=list(df_iso.loc[df_iso['actor_id'].isnull(), 'country']):
    print(f"Countries not found: {not_found}")

In [ ]:
df.head()

,country,year,emissions,total_emissions
0,Canada,1990,480.394916,4.803949e+08
1,Mexico,1990,315.852733,3.158527e+08
2,US,1990,5275.397531,5.275398e+09
3,Argentina,1990,123.904223,1.239042e+08
4,Brazil,1990,217.067921,2.170679e+08


In [ ]:
raw.head(2)

,Million tonnes of carbon dioxide equivalent,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2015,2016,2017,2018,2019,2020,2021,2021.1,2011-21,2021.2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Canada,480.394916,463.725716,479.595415,477.188334,494.227769,515.442121,525.159938,539.822963,557.345635,...,626.195347,609.108154,629.367081,642.112663,639.321238,582.243625,595.422692,0.025437,-0.001249,0.015276


Create a bot that generates new parsing code from the current repository

In [ ]:
load_dotenv()

True

In [ ]:
from langchain.document_loaders import TextLoader

root_dir = '/content/OpenClimate/harmonize/scripts/'

docs = []
count = 0
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith('.py') and '/.venv/' not in dirpath:
            try: 
                count += 1
                loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
                docs.extend(loader.load_and_split())
            except Exception as e: 
                pass
print(f'{len(docs)}')

185


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1600, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
print(f"{len(texts)}")

467


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=None, openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None)

In [ ]:
DEEPLAKE_ACCOUNT_NAME='retroam'

In [ ]:
from langchain.vectorstores import DeepLake

db = DeepLake.from_documents(texts, embeddings, dataset_path=f"hub://{DEEPLAKE_ACCOUNT_NAME}/ocean-code")
db

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/katmphego/ocean-code
hub://katmphego/ocean-code loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:15<00:00
-

Dataset(path='hub://katmphego/ocean-code', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (467, 1536)  float32   None   
    ids      text     (467, 1)      str     None   
 metadata    json     (467, 1)      str     None   
   text      text     (467, 1)      str     None   


In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 20
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 20

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name='gpt-4') # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [84]:
raw_ex = raw.head(2).to_markdown()
clean_ex = df.head(2).to_markdown()

In [87]:
prompt = (f"""
I have two datasets that have different data schemas and I want to harmonize 
them so that they use the same one. Can you provide python code that 
would turn Data Schema B into the same structure as Data Schema A. Use prior code
as a starting point
    
    Data A examples is:

    {clean_ex}
    
    Data  B example is:

    {raw_ex}
    
    Provide the python code explaining what each part does in its transformation? Be concise
    
    Thank you
""")

In [88]:
questions = [
"""
what does the codebase do
""",
"can you write a single code file that replaces all the code?",
prompt
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: 
what does the codebase do
 

**Answer**: The codebase provided consists of multiple scripts that primarily perform the following tasks:

1. Process and harmonize various climate-related datasets, such as emissions data, climate targets, and other related information from different sources, like CDP (Carbon Disclosure Project), World Bank, UNFCCC (United Nations Framework Convention on Climate Change), Climate Mayors, and others.

2. Extract relevant data, such as country codes, actor names, territories, and other information from these datasets.

3. Create output directories, process data, and store the results in CSV files.

4. Generate tables for publishers, datasources, and other metadata associated with the datasets.

The codebase makes use of Python libraries like pandas, numpy, and others to manipulate and process the data effectively. 

-> **Question**: can you write a single code file that replaces all the code? 

**Answer**: It's not possible to create a sing